<a href="https://colab.research.google.com/github/KNUckle-llm/experiments/blob/main/RAG_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 개발 환경 확인
- GPU 환경을 확인하는 명령어 nivida-smi, L4 혹은 A100 인지 확인
- 고용량 RAM 용량 확인

In [1]:
!free -h

               total        used        free      shared  buff/cache   available
Mem:            52Gi       816Mi        47Gi       1.0Mi       4.4Gi        51Gi
Swap:             0B          0B          0B


In [2]:
!nvidia-smi

Tue Apr  1 14:16:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   37C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 필요한 모듈이나 라이브러리 설치

- 해당 셀에서 필요한 모듈 혹은 라이브러리를 설치(Ctrl + M + A)

In [3]:
!pip install -q chromadb whoosh langchain sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 77.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 99.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.9 MB/s eta 0:0

# VectorStore 로드
- 프로젝트에 사용되고 있는 VectorStore 로드

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# 저장된 결과 보기
# 1. 필요한 모듈 임포트
from chromadb import PersistentClient

# 2. ChromaDB 클라이언트 연결
persist_dir = "/content/drive/MyDrive/chroma_index"  # 실제 저장된 경로로 설정
client = PersistentClient(path=persist_dir)

# 저장된 컬렉션 불러오기 (컬렉션 이름은 실제 생성한 이름으로!)
collection = client.get_or_create_collection(name="knu_collection")

# 전체 문서 + 메타데이터 + 임베딩 가져오기
results = collection.get(include=["documents", "metadatas", "embeddings"])

# 결과 출력
print(f"✅ 저장된 총 청크 수: {len(results['documents'])}")
for i in range(len(results['documents'])):
    print(f"🆔 ID: {results['ids'][i]}")
    print(f"📄 청크 내용: {results['documents'][i][:200]}...")  # 너무 길면 앞부분만
    print(f"📎 메타데이터: {results['metadatas'][i]}")
    print(f"🔢 임베딩 벡터 (일부): {results['embeddings'][i][:5]}...")  # 임베딩 앞부분 5개만 보기
    print("-" * 80)

✅ 저장된 총 청크 수: 24
🆔 ID: Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_0
📄 청크 내용: - 7 -소프트웨어전공 졸업논문 규정 제1조(목적) 이 내규는 공주대학교 학칙(이하 “학칙”이라 함) 제94조 및 공주대학교 학사운영 규정(이하 “학사규정 ”이라 함) 제12장에 의거 소프트웨어전공 , 컴퓨터소프트웨어공학전공 및 멀티미디어공학전공의 졸업논문에 관한 세부사항을 규정함을 목적으로 한다. 제2조(졸업논문 제출자격 ) ① 최종학기 등록을 필한 자라...
📎 메타데이터: {'category': '규정자료실', 'date': '2025-04-01T18:16:21.214686+09:00', 'department': 'department (학부)/Cheonan College of Engineering (천안공과대학)/Software Department (소프트웨어학과)', 'file_name': '소프트웨어전공 졸업규정', 'source_type': 'pdf', 'source_url': 'https://sw.kongju.ac.kr/synap/skin/doc.html?fn=ZD1180_1426_208960_3&rs=/synap/result/bbs/1426'}
🔢 임베딩 벡터 (일부): [-0.02203827  0.06772523  0.01900545  0.01029711 -0.0526075 ]...
--------------------------------------------------------------------------------
🆔 ID: Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_1
📄 청크 내용: 4 이전까지 발표하고 , 종강전까지 인쇄본을 제출하여야한다 ② 단독연구만 을 인정한다 . ③ 지도교수는 졸업논문의 자료준비 , 작성 및 발표 등에 관한 사항을 지도한다 . ④ 졸업논문

# Advanced RAG
- Naive RAG에서 Retrieval 처리를 더 강화한 RAG 패러다임
![Advanced RAG](https://miro.medium.com/v2/resize:fit:1400/0*d8aZGfRkfcFVf4jA.png)

## **Retrieval**

- BM25 + Dense Vector Index + Reciprocal Rank Fusion -> Hybrid Search

### [**Okapi BM25**](https://en.wikipedia.org/wiki/Okapi_BM25)
- _BM_ 은 best matching의 약어
- 확률적 정보 검색(IR: Information Retrieval)에서 널리 쓰이는 문서 랭킹 함수
1. 쿼리에서 등장하는 단어가 문서에서 얼마나 자주 등장하는지 **(TF: Term Frequency)**
2. 그 단어가 전체 문서에서 얼마나 희귀한지 **(IDF: Inverse Document Frequency)**
3. 문서의 길이에 따라 가중치 부여 (길이가 긴 문서는 점수를 낮추는 방식)

</br>
$$
\text{score}(D, Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot
\frac{f(q_i, D) \cdot (k_1 + 1)}
{f(q_i, D) + k_1 \cdot \left(1 - b + b \cdot \frac{|D|}{\text{avgdl}}\right)}
$$
</br>

- $f(q_i, D)$ : 문서 $D$ 내의 특정 단어 $q_i$가 나타내는 횟수
- $|D|$ : 문서 $D$의 길이
- avgdl : 전체 문서의 평균 길이
- $k_1$과 $b$ : hyper parameters
  - $k_1$ : 키워드의 빈도에 따른 점수 영향력 억제, 무분별한 키워드 빈도 수 증가에 따른 중요도 향상 억제한다. (default 1.2 ~ 2.0)
  - $b$ : 문서 길이에 대한 중요도 제한, 짧은 문서에 특정 단어가 존재하면 중요도를 높게 평가한다. (default 0.75)

</br>
$$
\text{IDF}(q_i) = \ln \left( \frac{N - n(q_i) + 0.5}{n(q_i) + 0.5} + 1 \right)
$$
</br>

- $N$ : 전체 문서 수
- $n(q_i)$ : 단어 $q_i$를 포함하는 문서의 수
- IDF(Inverse Document Frequency)는 특정 단어 $q_i$가 모든 문서에 존재한다면 중요하지 않다고 여기고 특정 문서에만 존재하는 단어라면 중요하다고 평가한다. **IDF가 낮으면 중요도가 낮고, IDF가 높으면 중요도가 높다.**



In [6]:
# BM25 from Scratch
import math
from collections import Counter

class BM25:
  def __init__(self, corpus, k1=1.2, b=0.75):
    self.corpus = corpus
    self.k1 = k1
    self.b = b
    self.N = len(corpus)
    self.avgdl = sum(len(doc) for doc in corpus) / self.N
    self.df = self._calculate_df()
    self.idf = self._calculate_idf()

  # Python에서 함수 앞에 _는 private이라는 관용 표현, 외부에서 호출하지 않는다는 의미
  ## Document Frequency
  def _calculate_df(self):
    df = {}
    for doc in self.corpus:
      for word in set(doc):
        # 딕셔너리 df에 word 키가 이미 있으면 +1, 없으면 0 + 1로 새로 추가
        df[word] = df.get(word, 0) + 1
    return df

  #Inverted Document Frequency
  def _calculate_idf(self):
    idf = {}
    for term, df in self.df.items():
        idf[term] = math.log((self.N - df + 0.5) / (df + 0.5) + 1)
    return idf

  def get_scores(self, query, doc_index):
    doc = self.corpus[doc_index]
    doc_len = len(doc)
    tf = Counter(doc)
    score = 0.0

    for q in query:
        if q not in tf:
            continue
        idf = self.idf.get(q, 0)
        freq = tf[q]
        denom = freq + self.k1 * (1 - self.b + self.b * (doc_len / self.avgdl))
        score += idf * ((freq * (self.k1 + 1)) / denom)
    return score

  def get_top_k(self, query, k=3):
    scores = [(i, self.get_scores(query, i)) for i in range(self.N)]
    return sorted(scores, key=lambda x: x[1], reverse=True)[:k]

In [7]:
corpus = [
    "the cat in the hat".split(),
    "the quick brown fox".split(),
    "the lazy dog and the fox".split()
]

query = "fox and dog"

bm25 = BM25(corpus)
top_docs = bm25.get_top_k(query.split())

for idx, score in top_docs:
    print(f"Doc {idx}: {corpus[idx]} → Score: {score:.4f}")

Doc 2: ['the', 'lazy', 'dog', 'and', 'the', 'fox'] → Score: 2.2478
Doc 1: ['the', 'quick', 'brown', 'fox'] → Score: 0.5119
Doc 0: ['the', 'cat', 'in', 'the', 'hat'] → Score: 0.0000


### **Whoosh Index**
- Sparse Index
- Whoosh는 **역색인(Inverted Index)**을 만드는 전통적인 키워드 기반 검색엔진
1. 문서 텍스트를 토큰화 (단어 단위로 분리)
2. 각 단어가 등장한 문서 ID와 위치를 인덱싱
3. 질의(query)가 들어오면, 일치하는 단어를 포함한 문서를 빠르게 찾아냄
4. BM25 등의 스코어링 함수를 사용해서 관련성을 계산함



In [8]:
# 📌 1. 필요한 모듈 임포트
from whoosh.index import create_in, open_dir, exists_in
from whoosh.fields import Schema, TEXT, ID
import os

from chromadb import PersistentClient

# 📌 2. ChromaDB 클라이언트 연결
persist_dir = "/content/drive/MyDrive/chroma_index"  # 실제 저장된 경로
client = PersistentClient(path=persist_dir)
collection = client.get_or_create_collection(name="knu_collection")

# 📌 3. Chroma 내 전체 문서 불러오기
results = collection.get(include=["documents", "metadatas"])  # embeddings는 필요 없음
doc_ids = results["ids"]
doc_texts = results["documents"]
print(f"✅ 저장된 총 청크 수: {len(doc_texts)}")

# Whoosh 인덱스 경로 설정
whoosh_index_dir = "/content/drive/MyDrive/whoosh_index"
schema = Schema(doc_id=ID(stored=True), content=TEXT(stored=True))

# 📌 4. Whoosh 인덱스 열기 또는 새로 생성
if not exists_in(whoosh_index_dir):
    os.makedirs(whoosh_index_dir, exist_ok=True)
    ix = create_in(whoosh_index_dir, schema)
else:
    ix = open_dir(whoosh_index_dir)

writer = ix.writer()

# 📌 5. Whoosh에 이미 저장된 문서 ID 목록 확인
with ix.searcher() as searcher:
    existing_ids = set()
    for fields in searcher.all_stored_fields():
        existing_ids.add(fields["doc_id"])

# 📌 6. 새 문서만 선별
new_docs = [
    {"doc_id": doc_id, "content": doc_text}
    for doc_id, doc_text in zip(doc_ids, doc_texts)
    if doc_id not in existing_ids
]

print(f"🆕 새로 추가할 문서 수: {len(new_docs)}")

# 📌 7. 새 문서만 Whoosh에 추가
for doc in new_docs:
    writer.add_document(doc_id=doc["doc_id"], content=doc["content"])

writer.commit()
print("✅ Whoosh 인덱스 업데이트 완료!")

✅ 저장된 총 청크 수: 24
🆕 새로 추가할 문서 수: 0
✅ Whoosh 인덱스 업데이트 완료!


### **상호 순위 조합(Reciprocal Ranking Fusion, RRF)**
- 여러 개의 **랭킹 결과(rank lists)**를 결합해서 최종 랭킹을 만드는 방식

</br>
$$
\text{RRF}(D) = \sum_{i=1}^{n}
\frac{1}{k+rank_i(d)}
$$
</br>

- $d$: 문서
- $rank_i(d)$ : i번째 시스템에서 문서 d의 랭킹 위치 (1위면 1, 2위면 2)
- $k$: 순위 간 점수 차이를 부드럽게 만드는 역할 (보통 60을 많이 씀)

예를 들어,
- BM25 결과:
  - doc1: 1위
  - doc2: 2위
  - doc3: 3위
- ANN 결과:
  - doc3: 1위
  - doc2: 2위
  - doc4: 3위

→ RRF 점수 계산 (k=60):
```
doc1: 1 / (60 + 1)           = 0.016
doc2: 1 / (60 + 2) + 1 / (60 + 2) = 0.032
doc3: 1 / (60 + 3) + 1 / (60 + 1) = 0.032
doc4: 1 / (60 + 3)           = 0.016

```
→ 최종 순위: doc2 = doc3 > doc1 = doc4

- 서로 다른 검색 모델의 결과를 공정하게 섞고 싶을 때 사용
- 해당 실험에서는 Dense Vector Index(Chroma) + Sparse Index 결과를 RRF로 조합하고자 함.

In [9]:
from whoosh.qparser import QueryParser
from sentence_transformers import SentenceTransformer
from langchain.schema import Document
from collections import defaultdict

# HuggingFace 기반 임베딩 모델
embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

def hybrid_search_rrf(query: str, top_k=3, rrf_k=10):
    # 1. Whoosh 결과(BM25 이용)
    with ix.searcher() as searcher:
        parser = QueryParser("content", schema=ix.schema)
        whoosh_query = parser.parse(query)
        results = searcher.search(whoosh_query, limit=top_k)
        whoosh_results = [(r["doc_id"], r["content"]) for r in results]

    # 2. Chroma 결과
    q_embedding = embedding_model.encode([query]).tolist()[0]
    chroma_results_raw = collection.query(query_embeddings=[q_embedding], n_results=top_k)
    chroma_results = list(zip(chroma_results_raw["ids"][0], chroma_results_raw["documents"][0]))

    # 3. RRF 병합
    rank_scores = defaultdict(float)
    all_docs = []

    for rank, (doc_id, content) in enumerate(whoosh_results):
        rank_scores[doc_id] += 1 / (rrf_k + rank)
        all_docs.append((doc_id, content))

    for rank, (doc_id, content) in enumerate(chroma_results):
        rank_scores[doc_id] += 1 / (rrf_k + rank)
        all_docs.append((doc_id, content))

    # 중복 제거 후 스코어 기준 정렬
    seen = {}
    for doc_id, content in all_docs:
        if doc_id not in seen:
            seen[doc_id] = content

    ranked_docs = sorted(seen.items(), key=lambda x: -rank_scores[x[0]])
    return [Document(page_content=seen[doc_id], metadata={"score": rank_scores[doc_id], "id": doc_id}) for doc_id, _ in ranked_docs[:top_k]]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
results = hybrid_search_rrf("캡스톤 지원금 규정")
for r in results:
    print(f"[{r.metadata['id']}] {r.page_content} (score: {r.metadata['score']:.2f})")

[Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_14] 공목적 달성 시까지  유의 사항 : 귀하는 상기 동의를 거부할 수 있습니다 . 해당 수집 항목은 정보통신 ·방송 연구개발 수행에 반드시 필요한 사항으로 이에 대한 동의를 하지 않을 경우에는 정보통신 ·방송 연구개발 참여 등에 제한을 받으실 수 있습니다 .개인정보 수집ᆞ이용ᆞ제공 동의서 (수혜인력 ) □ 과제현황 사업명 대학디지털교육역량강화 –SW중심대학 과제명 SW중심대학(국립공주대) 총사업기간 2024.5.1.～2031.12.31. 주관기관국립공주대학교 산학협력단 총괄책임자 임재현 □ 수혜인력 소속부서 (학과 )성명 학년 주민등록번호개인정보 수집 및 이용에 대한 동의고유식별정 보 처리동의개인정보의 제3자 제공에 대한 동의개인정보 이용 동의 (자필서명 ) 홍길동2학년010101-1 √ √ √ 3학년 상기연구개발참여자는<별지1>의"개인정보및과세정보수집및이용","국가연구자번호처 리","개인정보의제3자제공"내용을읽고명확히이해하였으며이에동의합니다. 2025년3월일 주관기관명:국립공주대학교산 (score: 0.10)
[Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_15] 학협력단기관장:김송자 (직인 또는 인감 ) 한국연구재단부설정보통신기획평가원장 귀하  학생연구원 , 장학금 수혜자 , 해외교육지원 , 산학협력지원 등 동 사업의 사업비 지원을 받는 학생을 대상 으로 작성 주민번호는뒷자리첫번째까지기입하고,“정보이용동의”란에는본인이직접서명하여야합산학캡스톤디자인 전문가 자문보고서 산업체 전문가 (지도위원 )성 명 소속/직위 지도일시20. . . / 00:00~00:00 ( )시간차수 과제팀 (지도대상 )소속학과 팀명/대표학생 과제명 지도내용 산학캡스톤디자인 작품에 대한 기술지도 내용, 지도성과 , 추후 기술지도 방안 등을 기술 

# Evaluation

In [11]:
# 평가용 QA dataset
qa_pairs = [
    {
        "query": "소프트웨어전공 졸업하려면 어떤 조건이 필요해?",
        "answer_ids": [
            "Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_0",
            "Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_1",
            "Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_2",
            "Software Department (소프트웨어학과)_소프트웨어전공 졸업규정_chunk_3"
        ]
    },
    {
        "query": "산학캡스톤디자인 지원금은 얼마까지 받을 수 있어?",
        "answer_ids": [
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_1"
        ]
    },
    {
        "query": "산학캡스톤디자인 신청서 제출은 어떻게 해?",
        "answer_ids": [
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_4",
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_6"
        ]
    },
    {
        "query": "산학캡스톤디자인 계획서 마감일은 언제야?",
        "answer_ids": [
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_3"
        ]
    },
    {
        "query": "산학캡스톤디자인 과제는 어떤 주제로 구성되어 있어?",
        "answer_ids": [
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_18",
            "Software Department (소프트웨어학과)_SW중심대학사업 2025학년도 1학기_산학캡스톤디자인 지원 공고문_chunk_19"
        ]
    }
]

In [12]:
def evaluate_retriever(qa_dataset, search_fn, top_k=5):
    from collections import defaultdict
    results = []

    for item in qa_dataset:
        query = item["query"]
        ground_truth_ids = set(item["answer_ids"])

        retrieved_docs = search_fn(query, top_k=top_k)
        retrieved_ids = [doc.metadata["id"] for doc in retrieved_docs]

        hit = int(any(doc_id in ground_truth_ids for doc_id in retrieved_ids))
        recall = len(ground_truth_ids.intersection(retrieved_ids)) / len(ground_truth_ids)

        # MRR 계산
        mrr = 0
        for rank, doc_id in enumerate(retrieved_ids):
            if doc_id in ground_truth_ids:
                mrr = 1 / (rank + 1)
                break

        results.append({
            "query": query,
            "hit@k": hit,
            "recall@k": recall,
            "mrr@k": mrr
        })

    return results

In [13]:
import pandas as pd

# 평가 실행
eval_results = evaluate_retriever(qa_pairs, hybrid_search_rrf, top_k=5)

# 데이터프레임으로 보기 좋게 출력
df = pd.DataFrame(eval_results)
print("📊 개별 질문별 평가 결과:")
print(df.to_markdown(index=False))

# 평균 성능 출력
print("\n📈 평균 성능:")
print(df[["hit@k", "recall@k", "mrr@k"]].mean())

📊 개별 질문별 평가 결과:
| query                                              |   hit@k |   recall@k |   mrr@k |
|:---------------------------------------------------|--------:|-----------:|--------:|
| 소프트웨어전공 졸업하려면 어떤 조건이 필요해?      |       1 |        0.5 |     0.5 |
| 산학캡스톤디자인 지원금은 얼마까지 받을 수 있어?   |       0 |        0   |     0   |
| 산학캡스톤디자인 신청서 제출은 어떻게 해?          |       0 |        0   |     0   |
| 산학캡스톤디자인 계획서 마감일은 언제야?           |       0 |        0   |     0   |
| 산학캡스톤디자인 과제는 어떤 주제로 구성되어 있어? |       0 |        0   |     0   |

📈 평균 성능:
hit@k       0.2
recall@k    0.1
mrr@k       0.1
dtype: float64
